referenfce: https://github.com/ukairia777/finance_sentiment_corpus

한국어 금융 뉴스 감성분석(긍정, 부정 분류)

In [1]:
!pip install transformers 
!pip install datasets

In [2]:
# 한국어 금융 뉴스 데이터 로드 >> 데이터 프레임에 저장 

import numpy as np 
import pandas as pd 

In [6]:
df = pd.read_excel('data\Training.xlsx')


In [7]:
print('샘플의 개수:', len(df))

샘플의 개수: 51630


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51630 entries, 0 to 51629
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  51630 non-null  int64 
 1   연령          51630 non-null  object
 2   성별          51630 non-null  object
 3   상황키워드       51630 non-null  object
 4   신체질환        51630 non-null  object
 5   감정_대분류      51630 non-null  object
 6   감정_소분류      51630 non-null  object
 7   사람문장1       51630 non-null  object
 8   시스템문장1      51630 non-null  object
 9   사람문장2       51630 non-null  object
 10  시스템문장2      51630 non-null  object
 11  사람문장3       42695 non-null  object
 12  시스템문장3      42695 non-null  object
dtypes: int64(1), object(12)
memory usage: 5.1+ MB


In [9]:
df = df[['감정_대분류', '사람문장1']]
df.head()

,감정_대분류,사람문장1
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


In [10]:
df.columns=['labels','sentence']
df.head()

,labels,sentence
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


In [11]:
df.head(3)

,labels,sentence
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...


In [12]:
df['labels'].value_counts()
# 중립(neural), 긍정(positive), 부정(negative)
# 정수 인코딩 (0,1,2) 변환 필요

labels
불안    9320
분노    9160
상처    9143
슬픔    9125
당황    8756
기쁨    6126
Name: count, dtype: int64

In [14]:
df['labels'].unique()

array(['분노', '기쁨', '불안', '당황', '슬픔', '상처'], dtype=object)

In [15]:
df['labels'] = df['labels'].replace(['분노','기쁨','불안', '당황', '슬픔', '상처'], [0,1,2,3,4,5])

In [16]:
df.head(3)

,labels,sentence
0,0,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,0,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,0,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...


In [21]:
df.head(10)

,labels,sentence
0,0,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,0,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,0,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,0,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,0,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
5,0,직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.
6,0,성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.
7,1,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.
8,2,졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.
9,1,요즘 직장생활이 너무 편하고 좋은 것 같아!


In [57]:
df[df['sentence'].duplicated()]

,labels,sentence
32758,0,남편 때문에 너무 속상해.
37076,3,나 걱정이 있어.
37675,1,오늘 감사한 일이 있었어.
37919,5,전부 언니는 공부를 잘한다고 언니한테만 잘해주고 나는 무시해.
38097,4,친구 집에 다녀왔어.
38533,4,나 좀 기분 나쁜 일이 있어.
38648,3,너무 힘들어.
38711,3,너무 외롭다.
38942,3,나 좀 잘못한 일이 있어.
39252,2,나 요즘 너무 힘들어.


In [58]:
df.drop_duplicates(subset = ['sentence'], inplace = True)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51601 entries, 0 to 51629
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   labels    51601 non-null  int64 
 1   sentence  51601 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [60]:
df.to_csv('Training.csv', index=False)

In [19]:
# load_dataset 모듈 활용 

from datasets import load_dataset

In [61]:
all_data = load_dataset(
            "csv",
            data_files={
                "train":"Training.csv"
            },
        )

# 현재 train에 모든 데이터가 저장됨 
# load_dataset() 허깅페이스 사에서 만든 transformer library 

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/SBAUser/.cache/huggingface/datasets/csv/default-0f449d9d34e78021/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
all_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'sentence'],
        num_rows: 51601
    })
})

In [63]:
# dataset 모듈 활용, train/test set 분류 
# train_test_split()

cs = all_data['train'].train_test_split(0.2)
# 8:2 비율로 훈련 데이터, 검증 데이터 구분, 저장 
train_cs = cs['train']
test_cs = cs['test']

In [64]:
train_cs

Dataset({
    features: ['labels', 'sentence'],
    num_rows: 41280
})

In [65]:
test_cs

Dataset({
    features: ['labels', 'sentence'],
    num_rows: 10321
})

In [66]:
# 검증 데이터 생성하기 
# 훈련용 데이터를 다시 8:2로 구분, 저장 

cs = train_cs.train_test_split(0.2)
train_cs = cs['train']
valid_cs = cs['test']

In [67]:
# 훈련 데이터 
print(train_cs)
print()
print(valid_cs)
print()
print(test_cs)

Dataset({
    features: ['labels', 'sentence'],
    num_rows: 33024
})

Dataset({
    features: ['labels', 'sentence'],
    num_rows: 8256
})

Dataset({
    features: ['labels', 'sentence'],
    num_rows: 10321
})


In [68]:
print('두번째 샘플 출력:', train_cs['sentence'][1])
print('두번째 샘플의 레이블 출력:', train_cs['labels'][1])

두번째 샘플 출력: 내 친구들 모두 취직했는데 나만 백수여서 슬퍼.
두번째 샘플의 레이블 출력: 3


데이터 셋 전처리

In [69]:
import numpy as np 
import pandas as pd
import random 
import time
import datetime 
from tqdm import tqdm 

import csv 
import os 

import tensorflow as tf 
import torch 

In [32]:
# BERT 사용하기 위한 모듈 불러오기 

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [33]:
# 패딩(padding) 위한 모듈 불러오기 

from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가지표 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [70]:
# BERT 구조 
# [CLS] sequence [SEP] 구조

# 훈련 데이터, 검증 데이터, 테스트 데이터를 BERT 구조로 변환 

train_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', train_cs['sentence']))
validation_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', valid_cs['sentence']))
test_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', test_cs['sentence']))

In [71]:
train_labels = train_cs['labels']
validation_labels = valid_cs['labels']
test_labels = test_cs['labels']

In [72]:
test_sentences[:5]

['[CLS]큰아들이 원해서 없는 살림에 무리해서 유학까지 보내줬는데 회사에 다닐 생각을 안 해.[SEP]',
 '[CLS]자녀들이 매번 나를 챙겨주고 애쓰니까 너무 고맙네.[SEP]',
 '[CLS]노화와 신체장애로 행동 실천은 어려워져서 안절부절못하게 돼.[SEP]',
 '[CLS]직장을 옮겼는데 겉돌고 있는 기분이야. 어떻게 해야 할지 모르겠어.[SEP]',
 '[CLS]만성 신경통으로 무릎이 안 좋은데 수술해도 완치가 어렵다니 너무 걱정스럽고 괴로워.[SEP]']

In [73]:
# 중립 = 0, 긍정 = 1, 부정 = 2 

test_labels[:5]

[0, 1, 2, 3, 2]

BERT 토큰나이저 이용한 전처리 

- BERT 사용하기 위해서는 tokenizer와 model이 반드시 mapping 되는 관계여야만 함 >> 아래의 이름에 들어가는 모델이름은 반드시 동일해야 함 
- BertTokenizer.from_pretrained('모델이름') 
- BertForSequenceClassification.from_pretrained('모델이름') 

- Tokenizer의 역할은 내부적으로 vocabulary를 갖고 있어서 정수 인코딩을 수행해주는 모듈

In [38]:
# kor-bert 중 하나니 'klue/bert-base'를 사용 
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [74]:
MAX_LEN = 128 
# pre-trained model : 최대 길이 512
# fine-tuning 할 때 데이터 셋 길이 128

def data_to_tensor(sentences, labels): 
  # 정수 인코딩 과정, 각 텍스트를 토큰화한 후에 vocabulary에 mapping 되는 정수 시퀀스로 변환 
  # 예) ['안녕하세요'] >> ['안','녕','하세요'] >> [231,52,45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences: padding 위한 모듈. 주어진 max_len(최대길이)위에 뒤에 0으로 채움 
  # 예) [231,52,45] >> [231,52,45,0,0,0]

  input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype='long', truncating='post', padding='post')
   
  attention_masks =[]

  for seq in input_ids:
     seq_mask = [float(i>0) for i in seq]
     attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks


In [75]:
# 훈련 데이터, 검증데이터, 테스트 데이터에 대해서 data_to_tensor 함수를 활용
# 정수 인코딩된 데이터, 레이블, 어텐션 마스크를 얻음 

train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

In [76]:
print(train_inputs[0])
print()
print(train_masks[0])

tensor([    2,   900,  2052,  1383,  6233,  1565,  2088,  4638,   723,  2073,
        10055,  2138,  5688,  2318,   858,  2051,    18,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],

In [77]:
tokenizer.decode([2])

'[CLS]'

In [78]:
tokenizer.decode([3])

'[SEP]'

In [79]:
tokenizer.decode([3678])

'그러나'

In [80]:
tokenizer.decode([3874])

'생산'

batch_size = 32 
- pytorch의 DataLoader이용, 변환 
- DataLoader: batch 단위로 데이터를 꺼내올 수 있도록 하는 모듈 

In [81]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [82]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [83]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [84]:
print('훈련 데이터 크기:', len(train_labels))
print('검증 데이터 크기:', len(validation_labels))
print('테스트 데이터 크기:', len(test_labels))

훈련 데이터 크기: 33024
검증 데이터 크기: 8256
테스트 데이터 크기: 10321


In [85]:
# GPU 정상 세팅이 되었는지 확인 

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3070


모델 로드하기 

- BERT 사용, 텍스트 분류 아키텍처
  - BertForSequenceClassification.from_pretrained('모델명') 
  - label 개수: num_labels

In [86]:
num_labels = 6

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels)
model.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [87]:
# 최적화(optimizer) 선택 

optimizer = AdamW(model.parameters(),
            lr= 2e-5,
            eps=1e-8)

c:\Users\SBAUser\Desktop\code\final\toon\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [88]:
# 학습 반복 횟수 설정 (epochs)

epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                num_warmup_steps = 0, 
                                num_training_steps = total_steps)

In [89]:
def format_time(elapsed): 
   elapsed_rounded = int(round(elapsed))
   return str(datetime.timedelta(seconds=elapsed_rounded)) 
   # hh:mm:ss 

In [90]:
def metrics(predictions, labels): 
   y_pred = predictions
   y_true = labels

   # 사용 가능한 메트릭들을 사용함 
   accuracy = accuracy_score(y_true, y_pred)
   f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0) 
   f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0) 
   f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
   
   # metric 결과에 대해서도 return 

   metrics = {'accuracy': accuracy, 
              'f1_macro': f1_macro_average,
              'f1_micro': f1_micro_average,
              'f1_weighted': f1_weighted_average
              }

   return metrics

모델 학습 

In [91]:
# random seed 값 설정 

seed_val = 777
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

for epoch_i in range(0, epochs): 
  print('---------- Epoch {:} / {:} ----------'.format(epoch_i + 1, epochs))
  t0 = time.time()
  total_loss = 0

  model.train() 

  for step, batch in tqdm(enumerate(train_dataloader)): 
    if step % 500 == 0 and not step == 0: 
      elapsed = format_time(time.time() - t0)
      print(' Batch {:>5,} of {:>5,}, Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
    batch = tuple(t.to(device) for t in batch)     
    b_input_ids, b_input_mask, b_labels = batch

    outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask = b_input_mask,
                    labels = b_labels
                    )  
    loss = outputs[0]     
    total_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 
    # threshold 초과시, gradient clipping 해줌 
    optimizer.step()
    scheduler.step()

    model.zero_grad()

  avg_train_loss = total_loss / len(train_dataloader)

  print("")
  print("Average training loss: {0:.4f}".format(avg_train_loss))
  print("Training epoch taken: {:}".format(format_time(time.time() - t0)))



---------- Epoch 1 / 10 ----------


500it [02:13,  3.74it/s]

 Batch   500 of 1,032, Elapsed: 0:02:14.


1000it [04:28,  3.77it/s]

 Batch 1,000 of 1,032, Elapsed: 0:04:28.


...

 Batch   500 of 1,032, Elapsed: 0:02:14.


1000it [04:28,  3.69it/s]

 Batch 1,000 of 1,032, Elapsed: 0:04:28.


1032it [04:36,  3.73it/s]


Average training loss: 0.1872
Training epoch taken: 0:04:37


검증 데이터에 대한 평가 

In [92]:
# eval() : 파이썬에서 문자열로 된 코드 실행 함수 
eval("1+2")

3

In [93]:
t0 = time.time()
model.eval() 
accum_logits, accum_label_ids = [], []

for batch in validation_dataloader: 
  batch = tuple(t.to(device) for t in batch) 
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad(): 
    outputs = model(b_input_ids, 
                    token_type_ids=None,
                    attention_mask = b_input_mask)
    
    # print(outputs[0])

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
      # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정 
      # 예를 들어 [3.5134, - 0.308765, -2.11316] >> 0
      accum_logits.append(np.argmax(b))

    for b in label_ids: 
      accum_label_ids.append(b)
  
accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print(accum_logits)
print(accum_label_ids)
print(results)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("f1_macro score: {0:.4f}".format(results['f1_macro']))
print("f1_micro score: {0:.4f}".format(results['f1_micro']))
print("f1_weighted score: {0:.4f}".format(results['f1_weighted']))

[2 1 5 ... 0 2 2]
[2 2 0 ... 3 0 2]
{'accuracy': 0.5604408914728682, 'f1_macro': 0.577665519751692, 'f1_micro': 0.5604408914728682, 'f1_weighted': 0.5604092619623735}
Accuracy: 0.5604
f1_macro score: 0.5777
f1_micro score: 0.5604
f1_weighted score: 0.5604


모델 저장과 모델 로드 

In [94]:
# 모델 저장 

torch.save(model.state_dict(), "BERT_emotion.pt")

In [95]:
# 모델 로드 

model.load_state_dict(torch.load("BERT_emotion.pt"))

<All keys matched successfully>

테스트 데이터에 대한 평가 

In [96]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [],[]

for step, batch in tqdm(enumerate(test_dataloader)):
  if step % 100 == 0 and not step == 0: 
    elapsed = format_time(time.time() - t0)
    print('Batch {:>5,} of {:>5,}, Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask = b_input_mask)
  logits = outputs[0]
  logits = logits.detach().cpu().numpy()  
  label_ids = b_labels.to('cpu').numpy()

  for b in logits: 
    accum_logits.append(np.argmax(b))

  for b in label_ids: 
    accum_label_ids.append(b)

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print(accum_logits)
print(accum_label_ids)
print(results)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("f1_macro score: {0:.4f}".format(results['f1_macro']))
print("f1_micro score: {0:.4f}".format(results['f1_micro']))
print("f1_weighted score: {0:.4f}".format(results['f1_weighted']))


101it [00:08, 11.80it/s]

Batch   100 of   323, Elapsed: 0:00:09.


201it [00:17, 11.70it/s]

Batch   200 of   323, Elapsed: 0:00:17.


301it [00:25, 11.89it/s]

Batch   300 of   323, Elapsed: 0:00:26.


323it [00:27, 11.74it/s]

[4 5 5 ... 5 5 1]
[4 5 3 ... 4 0 1]
{'accuracy': 0.5667086522623777, 'f1_macro': 0.5825823370818174, 'f1_micro': 0.5667086522623777, 'f1_weighted': 0.5661767176507392}
Accuracy: 0.5667
f1_macro score: 0.5826
f1_micro score: 0.5667
f1_weighted score: 0.5662


예측 

In [97]:
from transformers import pipeline
# pipeline : 정형화된 데이터 일 경우 사용 

In [98]:
pipe = pipeline('text-classification', model = model.cuda(), tokenizer = tokenizer, device=0, max_length=512, 
                return_all_scores=True, function_to_apply='softmax' )
pipe

# return_all_scores=True : 모든 라벨에 대한 확률값 반환 

c:\Users\SBAUser\Desktop\code\final\toon\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [99]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.011320759542286396}, {'label': 'LABEL_1', 'score': 0.3498938977718353}, {'label': 'LABEL_2', 'score': 0.42562052607536316}, {'label': 'LABEL_3', 'score': 0.06414298713207245}, {'label': 'LABEL_4', 'score': 0.005850953981280327}, {'label': 'LABEL_5', 'score': 0.14317083358764648}]]


In [100]:
result[0]

[{'label': 'LABEL_0', 'score': 0.011320759542286396},
 {'label': 'LABEL_1', 'score': 0.3498938977718353},
 {'label': 'LABEL_2', 'score': 0.42562052607536316},
 {'label': 'LABEL_3', 'score': 0.06414298713207245},
 {'label': 'LABEL_4', 'score': 0.005850953981280327},
 {'label': 'LABEL_5', 'score': 0.14317083358764648}]

In [101]:
result[0][0]

{'label': 'LABEL_0', 'score': 0.011320759542286396}

In [102]:
result[0][0]['label']

'LABEL_0'

In [103]:
print(result)

[[{'label': 'LABEL_0', 'score': 0.011320759542286396}, {'label': 'LABEL_1', 'score': 0.3498938977718353}, {'label': 'LABEL_2', 'score': 0.42562052607536316}, {'label': 'LABEL_3', 'score': 0.06414298713207245}, {'label': 'LABEL_4', 'score': 0.005850953981280327}, {'label': 'LABEL_5', 'score': 0.14317083358764648}]]


In [104]:
pipe = pipeline('text-classification', model = model.cuda(), tokenizer = tokenizer, device=0, max_length=512, 
                 function_to_apply='softmax' )

In [105]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)

[{'label': 'LABEL_2', 'score': 0.42562052607536316}]


In [106]:
label_dict = {'LABEL_0': '분노', 'LABEL_1':'기쁨', 'LABEL_2':'불안', 'LABEL_3': '당황', 'LABEL_4':'슬픔', 'LABEL_5':'상처'}

In [107]:
label_dict['LABEL_0']

'분노'

In [108]:
def prediction(text): 
   result = pipe(text)

   return [label_dict[result[0]['label']]]

In [109]:
prediction("매출이 급성장했다")

['기쁨']

In [110]:
prediction("챗지피 등장으로 기존 산업이 위협을 느끼고 있다")

['상처']

In [111]:
prediction("가상환경을 만드는 것은 정말 거지같아")

['분노']

In [112]:
prediction("코버트는 정말 쓰레기다")

['분노']

In [113]:
prediction("나 너무 신나")

['기쁨']

In [118]:
prediction("호밀이 귀여워서 깨물어 죽이고싶어")

c:\Users\SBAUser\Desktop\code\final\toon\lib\site-packages\transformers\pipelines\base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


['분노']

In [119]:
prediction("호밀이 너무 귀여워서 깨물어주고싶어")

['상처']

In [120]:
prediction("비가 갑자기 오네")

['불안']

In [121]:
prediction("아이고 깜짝이야")

['당황']